In [ ]:
import numpy as np
from src.utils import load_data
from src.transformations import simple_transform
import os
import torch
import torchvision
import torchvision.transforms as transforms
from src.transformations import pretrained_transform, normalized_simple_transform, simple_transform

In [ ]:
loader = load_data('./data/train', batch_size=512, shuffle=True, transform=simple_transform(), num_workers=2)

mean = 0.0
std = 0.0
count = 0

for data, _ in loader:
    batch_count = data.size(0) 
    data = data.view(batch_count, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    count += batch_count
mean = mean / count
std =  std / count
print("Mean ", mean)
print("Std ", std)


Mean  tensor([0.4789, 0.4723, 0.4305])
Std  tensor([0.1998, 0.1965, 0.1997])


In [5]:
print(count)

90000


In [6]:
dataset_path = './data/train'
preprocessed_path = './data/resized_normalized/train'

torch.set_num_threads(12)
os.makedirs(preprocessed_path, exist_ok=True)

dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=pretrained_transform())

for i, (img, label) in enumerate(dataset):
    save_path = os.path.join(preprocessed_path, f'img_{i}.pt')
    torch.save({'image': img, 'label': label}, save_path)

    if i % 1000 == 0:
        print(f"Saved {i} samples")


Saved 0 samples
Saved 100 samples
Saved 200 samples
Saved 300 samples
Saved 400 samples
Saved 500 samples
Saved 600 samples
Saved 700 samples
Saved 800 samples
Saved 900 samples
Saved 1000 samples
Saved 1100 samples
Saved 1200 samples
Saved 1300 samples
Saved 1400 samples
Saved 1500 samples
Saved 1600 samples
Saved 1700 samples
Saved 1800 samples
Saved 1900 samples
Saved 2000 samples
Saved 2100 samples
Saved 2200 samples
Saved 2300 samples
Saved 2400 samples
Saved 2500 samples
Saved 2600 samples
Saved 2700 samples
Saved 2800 samples
Saved 2900 samples
Saved 3000 samples
Saved 3100 samples
Saved 3200 samples
Saved 3300 samples
Saved 3400 samples
Saved 3500 samples
Saved 3600 samples
Saved 3700 samples
Saved 3800 samples
Saved 3900 samples
Saved 4000 samples
Saved 4100 samples
Saved 4200 samples
Saved 4300 samples
Saved 4400 samples
Saved 4500 samples
Saved 4600 samples
Saved 4700 samples
Saved 4800 samples
Saved 4900 samples
Saved 5000 samples
Saved 5100 samples
Saved 5200 samples
Saved

In [ ]:
dataset_path = './data/train'
preprocessed_path = './data/normalized/train'

torch.set_num_threads(12)
os.makedirs(preprocessed_path, exist_ok=True)

dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=normalized_simple_transform())

for i, (img, label) in enumerate(dataset):
    save_path = os.path.join(preprocessed_path, f'img_{i}.pt')
    torch.save({'image': img, 'label': label}, save_path)

    if i % 1000 == 0:
        print(f"Saved {i} samples")

In [2]:
w = 0.002
w_str = f"{w}".replace("0.", "")
w_str

'002'

In [9]:
from src.models.our_model1 import OurModelBig, OurModel, OurModelSmall
from src.utils import load_data
import torch
from src.utils import load_data, evaluate_model
from src.model_trainer import ModelTrainer
from torchsummary import summary
from src.transformations import normalized_simple_transform
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
torch.manual_seed(123)
print(device)
torch.set_num_threads(14)

cpu


In [2]:
model = OurModel()
train_loader = load_data('./data/train', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type='adam',  
    learning_rate=0.001,
    log_file='trainlogtest.json',  
    save_dir='./saved_models/test',
    max_batches = 1
)

In [ ]:
trainer.train(1)

In [4]:
model = OurModelBig()
train_loader = load_data('./data/train', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type='adam',  
    learning_rate=0.001,
    log_file='training_log_ourmodel1bigger.json',  
    save_dir='./saved_models/ourmodel1bigger',
    max_batches = None
)

In [6]:
trainer.load_checkpoint()

Loaded logs from ./saved_models/ourmodel1bigger\training_log_ourmodel1bigger.json with last epoch 3
Loaded model with epoch 3 from ./saved_models/ourmodel1bigger\OurModelBig_3.pth


In [7]:
from src.utils import evaluate_model
test_loader = load_data('./data/test', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
evaluate_model(trainer.model, test_loader, device, max_batches=10)

1 2 3 4 5 6 7 8 9 10 

0.71796875

In [10]:
model = OurModelSmall()
train_loader = load_data('./data/train', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type='adam',  
    learning_rate=0.001,
    log_file='training_log_ourmodel1.json',  
    save_dir='./saved_models/ourmodel1',
    max_batches = None
)

In [11]:
trainer.load_checkpoint()

Loaded logs from ./saved_models/ourmodel1\training_log_ourmodel1.json with last epoch 10
Loaded model with epoch 10 from ./saved_models/ourmodel1\OurModelSmall_10.pth


In [12]:
from src.utils import evaluate_model
test_loader = load_data('./data/test', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
evaluate_model(trainer.model, test_loader, device, max_batches=10)

1 2 3 4 5 6 7 8 9 10 

0.6609375

In [3]:
trainer.train(1)

Batch count 0
Epoch 1/1, Loss: 3.0108, Accuracy: 0.0781, F1-score: 0.0558
Training log saved to ./saved_models/test\trainlogtest.json
Model saved to ./saved_models/test\OurModel_1.pth
